## SQL Alchemy run through

In the previous notebook, we made an RDS Instance that contains all of our tables. In this subsection, we will go over retrieving this information using SQL Alchemy

## Making the engine

In [1]:
from sqlalchemy import create_engine, select
from sqlalchemy.dialects.postgresql import psycopg2
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
import psycopg2
import sqlalchemy

In [2]:
import boto3
import pandas as pd

In [3]:
int(sqlalchemy.__version__[0]) >= 1

True

In [4]:
from sqlalchemy.sql import and_, or_

In [5]:
from sqlalchemy import func, text, case

In [6]:
from pprint import pprint

### first get the rds instance we deployed earlier

In [7]:
rds = boto3.client('rds')

In [8]:
def getAddressAndPort(instance_identifier):
    import boto3
    rds_client = boto3.client('rds')

    instance =filter(
        lambda instance: instance['DBInstanceIdentifier'] == instance_identifier,
                           rds.describe_db_instances()['DBInstances'])
    endpoint= list(instance)[0]['Endpoint']
    address, port = endpoint['Address'], endpoint['Port']
    return address, port



In [9]:
getAddressAndPort('cfsdb')

('cfsdb.cs4yyoqmq4gl.us-west-1.rds.amazonaws.com', 5432)

In [10]:
instances = list(filter(lambda instance: instance['DBInstanceStatus']=='available', rds.describe_db_instances()['DBInstances']))

In [11]:
instance = instances[0]

In [12]:
instances

[{'AllocatedStorage': 5,
  'AutoMinorVersionUpgrade': True,
  'AvailabilityZone': 'us-west-1c',
  'BackupRetentionPeriod': 5,
  'CACertificateIdentifier': 'rds-ca-2015',
  'CopyTagsToSnapshot': False,
  'DBInstanceClass': 'db.m4.large',
  'DBInstanceIdentifier': 'cfsdb',
  'DBInstanceStatus': 'available',
  'DBName': 'cfsdb',
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres9.4',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSecurityGroups': [],
  'DBSubnetGroup': {'DBSubnetGroupDescription': 'default',
   'DBSubnetGroupName': 'default',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetAvailabilityZone': {'Name': 'us-west-1c'},
     'SubnetIdentifier': 'subnet-1d5ab444',
     'SubnetStatus': 'Active'},
    {'SubnetAvailabilityZone': {'Name': 'us-west-1a'},
     'SubnetIdentifier': 'subnet-e9b1188c',
     'SubnetStatus': 'Active'}],
   'VpcId': 'vpc-bba470de'},
  'DbInstancePort': 0,
  'DbiResourceId': 'db-MRSQSSQK446ID2RK4BSFQ52DWE',
  'Endpoint': {'Address':

In [13]:
address, port = instance['Endpoint']['Address'], instance['Endpoint']['Port']
db_name = instance['DBName']
username = instance['MasterUsername']
password = 'TestPassword'

In [14]:
address, db_name

('cfsdb.cs4yyoqmq4gl.us-west-1.rds.amazonaws.com', 'cfsdb')

In [15]:
engine = create_engine('postgresql+psycopg2://{username}:{password}@{add}:{port}/{db_name}'.format(
    username=username,
    password=password,
    add=address,
    port=port,
    db_name=db_name), echo=True)

In [16]:
conn = engine.connect()

2016-03-10 23:11:03,234 INFO sqlalchemy.engine.base.Engine select version()
2016-03-10 23:11:03,235 INFO sqlalchemy.engine.base.Engine {}
2016-03-10 23:11:03,292 INFO sqlalchemy.engine.base.Engine select current_schema()
2016-03-10 23:11:03,293 INFO sqlalchemy.engine.base.Engine {}
2016-03-10 23:11:03,356 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-03-10 23:11:03,357 INFO sqlalchemy.engine.base.Engine {}
2016-03-10 23:11:03,383 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-03-10 23:11:03,384 INFO sqlalchemy.engine.base.Engine {}
2016-03-10 23:11:03,440 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2016-03-10 23:11:03,441 INFO sqlalchemy.engine.base.Engine {}


In [ ]:
# at this point we can remove entry 0 that we used in the earlier notebook
conn.execute("""
             DELETE FROM ONLY transactions
             WHERE "SHIPMT_ID" = 0 """)

In [17]:
assert conn.execute("""
    SELECT  "SHIPMT_ID" 
    FROM transactions
    LIMIT 1
    """).fetchall()[0][0] == 1, "Failed to delete row"

2016-02-08 21:00:12,307 INFO sqlalchemy.engine.base.Engine 
    SELECT  "SHIPMT_ID" 
    FROM transactions
    LIMIT 1
    


INFO:sqlalchemy.engine.base.Engine:
    SELECT  "SHIPMT_ID" 
    FROM transactions
    LIMIT 1
    


2016-02-08 21:00:12,309 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [18]:
# first try some basic sql queries
# get hazardous or not hazardous
q = conn.execute("""
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             """
             )

2016-02-08 21:00:32,182 INFO sqlalchemy.engine.base.Engine 
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             


INFO:sqlalchemy.engine.base.Engine:
             SELECT 
             CASE WHEN "HAZMAT" = 'P' OR "HAZMAT" = 'H' THEN 'Hazardous' 
                  WHEN "HAZMAT" = 'N' THEN 'Not Hazardous' 
                  ELSE  'n/a' END AS "hazmat", 
             COUNT(1) 
             FROM transactions 
             GROUP BY 1
             


2016-02-08 21:00:32,183 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [19]:
q.fetchall()

[('Hazardous', 302979), ('Not Hazardous', 4244682)]

### Now that you have it working, use SqlAlchemy to show all the tables you currently have

MetaData allows you to store all the information about all the tables in your database in a convenient location you have to use the reflect method of MetaData to accomplish this.

In [20]:
meta = MetaData()

In [21]:
meta.reflect(engine)

2016-02-08 21:01:09,735 INFO sqlalchemy.engine.base.Engine SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


INFO:sqlalchemy.engine.base.Engine:SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


2016-02-08 21:01:09,736 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-02-08 21:01:09,797 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in ('r', 'v', 'm', 'f')
        


...

INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


2016-02-08 21:01:12,093 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        


INFO:sqlalchemy.engine.base.Engine:
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        


2016-02-08 21:01:12,094 INFO sqlalchemy.engine.base.Engine {'table_oid': 16442}


INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


In [29]:
for table in meta.tables:
    print(table)

transactions_joined
FIPS_States
transactions
naics
cfs_areas
transport_mode
state_latlon
sctg


In [30]:
transactions = meta.tables['transactions']

state_latlon = meta.tables['state_latlon']

cfs_areas    = meta.tables['cfs_areas']
FIPS_States  = meta.tables['FIPS_States']
transport_mode = meta.tables['transport_mode']
sctg         = meta.tables['sctg']
transactions_joined = meta.tables['transactions_joined']

In [31]:
s = select([sctg])
r = conn.execute(s)

2016-02-08 21:06:04,257 INFO sqlalchemy.engine.base.Engine SELECT sctg."SCTG", sctg."Description" 
FROM sctg


INFO:sqlalchemy.engine.base.Engine:SELECT sctg."SCTG", sctg."Description" 
FROM sctg


2016-02-08 21:06:04,258 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [32]:
for row in r:
    print(row[sctg.c.Description], row[sctg.c.SCTG])
r.close()

Animals and Fish (live) 1
Cereal Grains (includes seed) 2
Agricultural Products (excludes Animal Feed, Cereal Grains, and Forage Products) 3
Animal Feed, Eggs, Honey, and Other Products of Animal Origin 4
Meat, Poultry, Fish, Seafood, and Their Preparations 5
Milled Grain Products and Preparations, and Bakery Products 6
Other Prepared Foodstuffs, and Fats and Oils 7
Alcoholic Beverages and Denatured Alcohol 8
Tobacco Products 9
Monumental or Building Stone 10
Natural Sands 11
Gravel and Crushed Stone (excludes Dolomite and Slate) 12
Other Non-Metallic Minerals not elsewhere classified 13
Metallic Ores and Concentrates 14
Coal 15
Crude Petroleum 16
Gasoline, Aviation Turbine Fuel, and Ethanol (includes Kerosene, and Fuel Alcohols) 17
Fuel Oils (includes Diesel, Bunker C, and Biodiesel) 18
Other Coal and Petroleum Products, not elsewhere classified 19
Basic Chemicals 20
Pharmaceutical Products 21
Fertilizers 22
Other Chemical Products and Preparations 23
Plastics and Rubber 24
Logs and O

### Joins

In [33]:
# join fips states with avg lat lon
for row in conn.execute(select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))):
    print(row)

2016-02-08 21:06:34,585 INFO sqlalchemy.engine.base.Engine SELECT state_latlon.state, state_latlon.latitude, state_latlon.longitude, "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM state_latlon, "FIPS_States" 
WHERE state_latlon.state = "FIPS_States"."Alpha code"


INFO:sqlalchemy.engine.base.Engine:SELECT state_latlon.state, state_latlon.latitude, state_latlon.longitude, "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM state_latlon, "FIPS_States" 
WHERE state_latlon.state = "FIPS_States"."Alpha code"


2016-02-08 21:06:34,587 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


('AL', 32.799, -86.8073, 'Alabama', 'AL', 1, 'State;\xa0counties')
('AK', 61.385, -152.2683, 'Alaska', 'AK', 2, 'State;\xa0boroughs')
('AS', 14.2417, -170.7197, 'American Samoa', 'AS', 60, 'Outlying area under U.S. sovereignty')
('AZ', 33.7712, -111.3877, 'Arizona', 'AZ', 4, 'State;\xa0counties')
('AR', 34.9513, -92.3809, 'Arkansas', 'AR', 5, 'State;\xa0counties')
('CA', 36.17, -119.7462, 'California', 'CA', 6, 'State;\xa0counties')
('CO', 39.0646, -105.3272, 'Colorado', 'CO', 8, 'State;\xa0counties')
('CT', 41.5834, -72.7622, 'Connecticut', 'CT', 9, 'State;\xa0counties')
('DE', 39.3498, -75.5148, 'Delaware', 'DE', 10, 'State;\xa0counties')
('DC', 38.8964, -77.0262, 'District of Columbia', 'DC', 11, 'Federal district[4]')
('FL', 27.8333, -81.717, 'Florida', 'FL', 12, 'State;\xa0counties')
('GA', 32.9866, -83.6487, 'Georgia', 'GA', 13, 'State;\xa0counties')
('HI', 21.1098, -157.5311, 'Hawaii', 'HI', 15, 'State;\xa0counties')
('ID', 44.2394, -114.5103, 'Idaho', 'ID', 16, 'State;\xa0count

In [34]:
FIPS_States.columns.get('Alpha code')

Column('Alpha code', TEXT(), table=<FIPS_States>)

### operators
SQL Alchemy converts python predicates to sql predicates. So you can describe everything in a pythonic sense.

In [35]:
print((FIPS_States.c.get('Alpha code') == 'KK').compile())
print(((FIPS_States.c.get('Alpha code') < 'CA') & ('NY' < FIPS_States.c.get('Alpha code'))).compile())

"FIPS_States"."Alpha code" = :Alpha code_1
"FIPS_States"."Alpha code" < :Alpha code_1 AND "FIPS_States"."Alpha code" > :Alpha code_2


In [36]:
transactions.columns.keys()

['SHIPMT_ID',
 'ORIG_STATE',
 'ORIG_MA',
 'ORIG_CFS_AREA',
 'DEST_STATE',
 'DEST_MA',
 'DEST_CFS_AREA',
 'NAICS',
 'QUARTER',
 'SCTG',
 'MODE',
 'SHIPMT_VALUE',
 'SHIPMT_WGHT',
 'SHIPMT_DIST_GC',
 'SHIPMT_DIST_ROUTED',
 'TEMP_CNTL_YN',
 'EXPORT_YN',
 'EXPORT_CNTRY',
 'HAZMAT',
 'WGT_FACTOR']

In [37]:
for row in conn.execute(select([transactions]).limit(10)).fetchall():
    print(row.items())

2016-02-08 21:08:41,987 INFO sqlalchemy.engine.base.Engine SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
 LIMIT %(param_1)s


INFO:sqlalchemy.engine.base.Engine:SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
 LIMIT %(param_1)s


2016-02-08 21:08:41,989 INFO sqlalchemy.engine.base.Engine {'param_1': 10}


INFO:sqlalchemy.engine.base.Engine:{'param_1': 10}


[('SHIPMT_ID', 1), ('ORIG_STATE', 25), ('ORIG_MA', 148), ('ORIG_CFS_AREA', '25-148'), ('DEST_STATE', 25), ('DEST_MA', 148), ('DEST_CFS_AREA', '25-148'), ('NAICS', 333), ('QUARTER', 2), ('SCTG', '35'), ('MODE', 14), ('SHIPMT_VALUE', 2178), ('SHIPMT_WGHT', 11), ('SHIPMT_DIST_GC', 14), ('SHIPMT_DIST_ROUTED', 17), ('TEMP_CNTL_YN', 'N'), ('EXPORT_YN', 'N'), ('EXPORT_CNTRY', 'N'), ('HAZMAT', 'N'), ('WGT_FACTOR', 208.5)]
[('SHIPMT_ID', 2), ('ORIG_STATE', 42), ('ORIG_MA', 428), ('ORIG_CFS_AREA', '42-428'), ('DEST_STATE', 6), ('DEST_MA', 41740), ('DEST_CFS_AREA', '06-41740'), ('NAICS', 311), ('QUARTER', 3), ('SCTG', '35'), ('MODE', 14), ('SHIPMT_VALUE', 344), ('SHIPMT_WGHT', 11), ('SHIPMT_DIST_GC', 2344), ('SHIPMT_DIST_ROUTED', 2734), ('TEMP_CNTL_YN', 'N'), ('EXPORT_YN', 'N'), ('EXPORT_CNTRY', 'N'), ('HAZMAT', 'N'), ('WGT_FACTOR', 193.3)]
[('SHIPMT_ID', 3), ('ORIG_STATE', 26), ('ORIG_MA', 220), ('ORIG_CFS_AREA', '26-220'), ('DEST_STATE', 47), ('DEST_MA', 314), ('DEST_CFS_AREA', '47-314'), ('NAI

In [38]:
# to get a list of dicts per row
get_row = lambda row: (dict(row.items()))
get_rows = lambda row_list: map(get_row, row_list)

list_of_dicts = [dict(row.items()) for row in conn.execute(select([transactions]).limit(10)).fetchall()]

2016-02-08 21:09:36,446 INFO sqlalchemy.engine.base.Engine SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
 LIMIT %(param_1)s


INFO:sqlalchemy.engine.base.Engine:SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
 LIMIT %(param_1)s


2016-02-08 21:09:36,448 INFO sqlalchemy.engine.base.Engine {'param_1': 10}


INFO:sqlalchemy.engine.base.Engine:{'param_1': 10}


### Conjunctions
Conjunctions can be used to perform and, or, not operations in sql.

In [39]:
# get all states that lie above the 35N, and less than -104 longitude

# first join the fips and state_latlons
j = FIPS_States.join(
    state_latlon, 
    onclause= FIPS_States.c.get("Alpha code") == state_latlon.c.state)

# now select from this join
s = (select([FIPS_States, state_latlon])
     .where(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        )
    ).select_from(j)
)

In [40]:
conn.execute(s).fetchall()

2016-02-08 21:13:19,115 INFO sqlalchemy.engine.base.Engine SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status", state_latlon.state, state_latlon.latitude, state_latlon.longitude 
FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = state_latlon.state 
WHERE state_latlon.latitude > %(latitude_1)s AND state_latlon.longitude < %(longitude_1)s


INFO:sqlalchemy.engine.base.Engine:SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status", state_latlon.state, state_latlon.latitude, state_latlon.longitude 
FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = state_latlon.state 
WHERE state_latlon.latitude > %(latitude_1)s AND state_latlon.longitude < %(longitude_1)s


2016-02-08 21:13:19,116 INFO sqlalchemy.engine.base.Engine {'latitude_1': 36.2, 'longitude_1': -104}


INFO:sqlalchemy.engine.base.Engine:{'latitude_1': 36.2, 'longitude_1': -104}


[('Alaska', 'AK', 2, 'State;\xa0boroughs', 'AK', 61.385, -152.2683),
 ('Colorado', 'CO', 8, 'State;\xa0counties', 'CO', 39.0646, -105.3272),
 ('Idaho', 'ID', 16, 'State;\xa0counties', 'ID', 44.2394, -114.5103),
 ('Montana', 'MT', 30, 'State;\xa0counties', 'MT', 46.9048, -110.3261),
 ('Nevada', 'NV', 32, 'State;\xa0counties', 'NV', 38.4199, -117.1219),
 ('Oregon', 'OR', 41, 'State;\xa0counties', 'OR', 44.5672, -122.1269),
 ('Utah', 'UT', 49, 'State;\xa0counties', 'UT', 40.1135, -111.8535),
 ('Washington', 'WA', 53, 'State;\xa0counties', 'WA', 47.3917, -121.5708),
 ('Wyoming', 'WY', 56, 'State;\xa0counties', 'WY', 42.7475, -107.2085)]

In [41]:
# now get those along with all the states in the bottom right corner of the US
# delinieated by below 35N and right of 88W
s = (select([FIPS_States, state_latlon])
     .where(or_(
        and_(
            state_latlon.c.latitude > 36.2,
            state_latlon.c.longitude < -104
        ),
        and_(
            state_latlon.c.latitude < 35,
            state_latlon.c.longitude > -88)
        )
    ).select_from(j)
)

In [42]:
pprint(str(s))

'SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", '
'"FIPS_States"."Numeric code", "FIPS_States"."Status", state_latlon.state, '
'state_latlon.latitude, state_latlon.longitude \n'
'FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = '
'state_latlon.state \n'
'WHERE state_latlon.latitude > :latitude_1 AND state_latlon.longitude < '
':longitude_1 OR state_latlon.latitude < :latitude_2 AND '
'state_latlon.longitude > :longitude_2'


In [43]:
pprint(str(select([text('sub."Alpha code"')]).select_from(s.alias('sub'))))

'SELECT sub."Alpha code" \n'
'FROM (SELECT "FIPS_States"."Name" AS "Name", "FIPS_States"."Alpha code" AS '
'"Alpha code", "FIPS_States"."Numeric code" AS "Numeric code", '
'"FIPS_States"."Status" AS "Status", state_latlon.state AS state, '
'state_latlon.latitude AS latitude, state_latlon.longitude AS longitude \n'
'FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = '
'state_latlon.state \n'
'WHERE state_latlon.latitude > :latitude_1 AND state_latlon.longitude < '
':longitude_1 OR state_latlon.latitude < :latitude_2 AND '
'state_latlon.longitude > :longitude_2) AS sub'


In [44]:
from sqlalchemy import desc
from sqlalchemy.sql import literal_column

In [45]:
# using text to select subqueries
# instead of text, we use literal_column so that we can use the label method
conn.execute(select([literal_column('sub."Alpha code"').label("alpha code")]).select_from(s.alias('sub'))
            .order_by(desc('alpha code'))).fetchall()

2016-02-08 21:15:21,933 INFO sqlalchemy.engine.base.Engine SELECT sub."Alpha code" AS "alpha code" 
FROM (SELECT "FIPS_States"."Name" AS "Name", "FIPS_States"."Alpha code" AS "Alpha code", "FIPS_States"."Numeric code" AS "Numeric code", "FIPS_States"."Status" AS "Status", state_latlon.state AS state, state_latlon.latitude AS latitude, state_latlon.longitude AS longitude 
FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = state_latlon.state 
WHERE state_latlon.latitude > %(latitude_1)s AND state_latlon.longitude < %(longitude_1)s OR state_latlon.latitude < %(latitude_2)s AND state_latlon.longitude > %(longitude_2)s) AS sub ORDER BY "alpha code" DESC


INFO:sqlalchemy.engine.base.Engine:SELECT sub."Alpha code" AS "alpha code" 
FROM (SELECT "FIPS_States"."Name" AS "Name", "FIPS_States"."Alpha code" AS "Alpha code", "FIPS_States"."Numeric code" AS "Numeric code", "FIPS_States"."Status" AS "Status", state_latlon.state AS state, state_latlon.latitude AS latitude, state_latlon.longitude AS longitude 
FROM "FIPS_States" JOIN state_latlon ON "FIPS_States"."Alpha code" = state_latlon.state 
WHERE state_latlon.latitude > %(latitude_1)s AND state_latlon.longitude < %(longitude_1)s OR state_latlon.latitude < %(latitude_2)s AND state_latlon.longitude > %(longitude_2)s) AS sub ORDER BY "alpha code" DESC


2016-02-08 21:15:21,934 INFO sqlalchemy.engine.base.Engine {'longitude_2': -88, 'latitude_2': 35, 'latitude_1': 36.2, 'longitude_1': -104}


INFO:sqlalchemy.engine.base.Engine:{'longitude_2': -88, 'latitude_2': 35, 'latitude_1': 36.2, 'longitude_1': -104}


[('WY',),
 ('WA',),
 ('VI',),
 ('UT',),
 ('SC',),
 ('PR',),
 ('OR',),
 ('NV',),
 ('MT',),
 ('MP',),
 ('ID',),
 ('GA',),
 ('FL',),
 ('CO',),
 ('AL',),
 ('AK',)]

#### Binding parameters

In [46]:
from sqlalchemy.sql import bindparam, func

In [47]:
get_results = lambda s, **kwargs: conn.execute(s, kwargs).fetchall()

In [48]:
get_results(
    select([FIPS_States])
        .where(or_(
              FIPS_States.c["Alpha code"] == bindparam('from_'),
              FIPS_States.c["Alpha code"] == bindparam('to_')
            )
        ),
    from_ = "CA",
    to_   = "TX"
)

2016-02-08 21:15:26,565 INFO sqlalchemy.engine.base.Engine SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(from_)s OR "FIPS_States"."Alpha code" = %(to_)s


INFO:sqlalchemy.engine.base.Engine:SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(from_)s OR "FIPS_States"."Alpha code" = %(to_)s


2016-02-08 21:15:26,566 INFO sqlalchemy.engine.base.Engine {'from_': 'CA', 'to_': 'TX'}


INFO:sqlalchemy.engine.base.Engine:{'from_': 'CA', 'to_': 'TX'}


[('California', 'CA', 6, 'State;\xa0counties'),
 ('Texas', 'TX', 48, 'State;\xa0counties')]

#### Functions

Functions are created using the func keyword.  These are translated to sql function queries.

func.myFunc(transactions.c.SHIPMT_WEIGHT) translates to 

    myFunc(transactions."SHIPMT_WEIGHT")

In [49]:
print(func.myFunc([transactions.c.SHIPMT_VALUE]).compile().params)

{'myFunc_1': [Column('SHIPMT_VALUE', BIGINT(), table=<transactions>)]}


In [50]:
conn.execute(select([func.max(transactions.c.SHIPMT_VALUE).label("Max Val")])).scalar()

2016-02-08 21:15:31,002 INFO sqlalchemy.engine.base.Engine SELECT max(transactions."SHIPMT_VALUE") AS "Max Val" 
FROM transactions


INFO:sqlalchemy.engine.base.Engine:SELECT max(transactions."SHIPMT_VALUE") AS "Max Val" 
FROM transactions


2016-02-08 21:15:31,004 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


521277800

#### Window functions
Chain over with functions

In [51]:
list(get_rows(get_results(select([transactions,
                                 func.sum(transactions.c.SHIPMT_VALUE).over(
                                    order_by=transactions.c.SHIPMT_ID)
                                  .label("SUM")
                                 ]).limit(10))))

2016-02-08 21:15:35,707 INFO sqlalchemy.engine.base.Engine SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR", sum(transactions."SHIPMT_VALUE") OVER (ORDER BY transactions."SHIPMT_ID") AS "SUM" 
FROM transactions 
 LIMIT %(param_1)s


INFO:sqlalchemy.engine.base.Engine:SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR", sum(transactions."SHIPMT_VALUE") OVER (ORDER BY transactions."SHIPMT_ID") AS "SUM" 
FROM transactions 
 LIMIT %(param_1)s


2016-02-08 21:15:35,709 INFO sqlalchemy.engine.base.Engine {'param_1': 10}


INFO:sqlalchemy.engine.base.Engine:{'param_1': 10}


[{'DEST_CFS_AREA': '25-148',
  'DEST_MA': 148,
  'DEST_STATE': 25,
  'EXPORT_CNTRY': 'N',
  'EXPORT_YN': 'N',
  'HAZMAT': 'N',
  'MODE': 14,
  'NAICS': 333,
  'ORIG_CFS_AREA': '25-148',
  'ORIG_MA': 148,
  'ORIG_STATE': 25,
  'QUARTER': 2,
  'SCTG': '35',
  'SHIPMT_DIST_GC': 14,
  'SHIPMT_DIST_ROUTED': 17,
  'SHIPMT_ID': 1,
  'SHIPMT_VALUE': 2178,
  'SHIPMT_WGHT': 11,
  'SUM': Decimal('2178'),
  'TEMP_CNTL_YN': 'N',
  'WGT_FACTOR': 208.5},
 {'DEST_CFS_AREA': '06-41740',
  'DEST_MA': 41740,
  'DEST_STATE': 6,
  'EXPORT_CNTRY': 'N',
  'EXPORT_YN': 'N',
  'HAZMAT': 'N',
  'MODE': 14,
  'NAICS': 311,
  'ORIG_CFS_AREA': '42-428',
  'ORIG_MA': 428,
  'ORIG_STATE': 42,
  'QUARTER': 3,
  'SCTG': '35',
  'SHIPMT_DIST_GC': 2344,
  'SHIPMT_DIST_ROUTED': 2734,
  'SHIPMT_ID': 2,
  'SHIPMT_VALUE': 344,
  'SHIPMT_WGHT': 11,
  'SUM': Decimal('2522'),
  'TEMP_CNTL_YN': 'N',
  'WGT_FACTOR': 193.3},
 {'DEST_CFS_AREA': '47-314',
  'DEST_MA': 314,
  'DEST_STATE': 47,
  'EXPORT_CNTRY': 'N',
  'EXPORT_YN': '

In [52]:
from sqlalchemy.sql import union

In [53]:
u = union(
    sctg.select().where(sctg.c.SCTG < 3),
    sctg.select().where(sctg.c.SCTG > 10)
    )

conn.execute(u).fetchall()

2016-02-08 21:15:40,195 INFO sqlalchemy.engine.base.Engine SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" < %(SCTG_1)s UNION SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" > %(SCTG_2)s


INFO:sqlalchemy.engine.base.Engine:SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" < %(SCTG_1)s UNION SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" > %(SCTG_2)s


2016-02-08 21:15:40,197 INFO sqlalchemy.engine.base.Engine {'SCTG_2': 10, 'SCTG_1': 3}


INFO:sqlalchemy.engine.base.Engine:{'SCTG_2': 10, 'SCTG_1': 3}


[(26, 'Wood Products'),
 (28, 'Paper or Paperboard Articles'),
 (35, 'Electronic and Other Electrical Equipment and Components, and Office Equipment'),
 (23, 'Other Chemical Products and Preparations'),
 (19, 'Other Coal and Petroleum Products, not elsewhere classified'),
 (32, 'Base Metal in Primary or Semi-Finished Forms and in Finished Basic Shapes'),
 (11, 'Natural Sands'),
 (38, 'Precision Instruments and Apparatus'),
 (41, 'Waste and Scrap (excludes of agriculture or food, see 041xx)'),
 (34, 'Machinery'),
 (14, 'Metallic Ores and Concentrates'),
 (18, 'Fuel Oils (includes Diesel, Bunker C, and Biodiesel)'),
 (13, 'Other Non-Metallic Minerals not elsewhere classified'),
 (39, 'Furniture, Mattresses and Mattress Supports, Lamps, Lighting Fittings, and Illuminated Signs'),
 (2, 'Cereal Grains (includes seed)'),
 (33, 'Articles of Base Metal'),
 (40, 'Miscellaneous Manufactured Products'),
 (24, 'Plastics and Rubber'),
 (12, 'Gravel and Crushed Stone (excludes Dolomite and Slate)'),

In [54]:
query_s = (select([func.count(transactions.c.SHIPMT_ID)])
           .where(transactions.c.SCTG).as_scalar())

In [55]:
conn.execute(select([query_s]))

2016-02-08 21:15:42,632 INFO sqlalchemy.engine.base.Engine SELECT (SELECT count(transactions."SHIPMT_ID") AS count_1 
FROM transactions 
WHERE transactions."SCTG") AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT (SELECT count(transactions."SHIPMT_ID") AS count_1 
FROM transactions 
WHERE transactions."SCTG") AS anon_1


2016-02-08 21:15:42,633 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-02-08 21:15:42,656 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


ProgrammingError: (psycopg2.ProgrammingError) argument of WHERE must be type boolean, not type text
LINE 3: WHERE transactions."SCTG") AS anon_1
              ^
 [SQL: 'SELECT (SELECT count(transactions."SHIPMT_ID") AS count_1 \nFROM transactions \nWHERE transactions."SCTG") AS anon_1']

In [56]:
conn.execute(u).fetchone()

2016-02-08 21:15:44,425 INFO sqlalchemy.engine.base.Engine SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" < %(SCTG_1)s UNION SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" > %(SCTG_2)s


INFO:sqlalchemy.engine.base.Engine:SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" < %(SCTG_1)s UNION SELECT sctg."SCTG", sctg."Description" 
FROM sctg 
WHERE sctg."SCTG" > %(SCTG_2)s


2016-02-08 21:15:44,427 INFO sqlalchemy.engine.base.Engine {'SCTG_2': 10, 'SCTG_1': 3}


INFO:sqlalchemy.engine.base.Engine:{'SCTG_2': 10, 'SCTG_1': 3}


(26, 'Wood Products')

##### Correlated Subqueries

When an enclosed query depends on a table from the enclosing context, it does not need to explicitly provide a FROM clause

In [57]:
stmt = (select([transactions, FIPS_States.c['Alpha code']])
        .select_from(
        transactions.join(FIPS_States, onclause=transactions.c.ORIG_STATE == FIPS_States.c["Numeric code"]))
        .where(FIPS_States.c['Alpha code'].like('C%'))
        )

In [58]:
enclosing_stmt = select([transactions.c.SCTG]).select_from(stmt).limit(10)

In [59]:
conn.execute(enclosing_stmt)

2016-02-08 21:15:53,280 INFO sqlalchemy.engine.base.Engine SELECT transactions."SCTG" 
FROM transactions, (SELECT transactions."SHIPMT_ID" AS "SHIPMT_ID", transactions."ORIG_STATE" AS "ORIG_STATE", transactions."ORIG_MA" AS "ORIG_MA", transactions."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions."DEST_STATE" AS "DEST_STATE", transactions."DEST_MA" AS "DEST_MA", transactions."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions."NAICS" AS "NAICS", transactions."QUARTER" AS "QUARTER", transactions."SCTG" AS "SCTG", transactions."MODE" AS "MODE", transactions."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN" AS "TEMP_CNTL_YN", transactions."EXPORT_YN" AS "EXPORT_YN", transactions."EXPORT_CNTRY" AS "EXPORT_CNTRY", transactions."HAZMAT" AS "HAZMAT", transactions."WGT_FACTOR" AS "WGT_FACTOR", "FIPS_States"."Alpha code" AS "Alpha cod

INFO:sqlalchemy.engine.base.Engine:SELECT transactions."SCTG" 
FROM transactions, (SELECT transactions."SHIPMT_ID" AS "SHIPMT_ID", transactions."ORIG_STATE" AS "ORIG_STATE", transactions."ORIG_MA" AS "ORIG_MA", transactions."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions."DEST_STATE" AS "DEST_STATE", transactions."DEST_MA" AS "DEST_MA", transactions."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions."NAICS" AS "NAICS", transactions."QUARTER" AS "QUARTER", transactions."SCTG" AS "SCTG", transactions."MODE" AS "MODE", transactions."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN" AS "TEMP_CNTL_YN", transactions."EXPORT_YN" AS "EXPORT_YN", transactions."EXPORT_CNTRY" AS "EXPORT_CNTRY", transactions."HAZMAT" AS "HAZMAT", transactions."WGT_FACTOR" AS "WGT_FACTOR", "FIPS_States"."Alpha code" AS "Alpha code" 
FROM transactions JO

2016-02-08 21:15:53,282 INFO sqlalchemy.engine.base.Engine {'Alpha code_1': 'C%', 'param_1': 10}


INFO:sqlalchemy.engine.base.Engine:{'Alpha code_1': 'C%', 'param_1': 10}


2016-02-08 21:15:53,311 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


ProgrammingError: (psycopg2.ProgrammingError) subquery in FROM must have an alias
LINE 2: FROM transactions, (SELECT transactions."SHIPMT_ID" AS "SHIP...
                           ^
HINT:  For example, FROM (SELECT ...) [AS] foo.
 [SQL: 'SELECT transactions."SCTG" \nFROM transactions, (SELECT transactions."SHIPMT_ID" AS "SHIPMT_ID", transactions."ORIG_STATE" AS "ORIG_STATE", transactions."ORIG_MA" AS "ORIG_MA", transactions."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions."DEST_STATE" AS "DEST_STATE", transactions."DEST_MA" AS "DEST_MA", transactions."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions."NAICS" AS "NAICS", transactions."QUARTER" AS "QUARTER", transactions."SCTG" AS "SCTG", transactions."MODE" AS "MODE", transactions."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN" AS "TEMP_CNTL_YN", transactions."EXPORT_YN" 

In [60]:
transactionsBetweenStates = (select([transactions_joined])
                                 .where(
                transactions_joined.c.orig_alpha_code==bindparam("orig_code"))
                                  .where(
                transactions_joined.c.dest_alpha_code==bindparam("dest_code"))
)


In [61]:
stmt = (select([transactionsBetweenStates.c['SHIPMT_VALUE']])
.select_from(transactionsBetweenStates.alias())
)



In [62]:
get_rows(conn.execute(stmt, orig_code="TX", dest_code="WA").fetchall())

2016-02-08 21:15:53,890 INFO sqlalchemy.engine.base.Engine SELECT "SHIPMT_VALUE" 
FROM (SELECT transactions_joined."SHIPMT_ID" AS "SHIPMT_ID", transactions_joined."ORIG_STATE" AS "ORIG_STATE", transactions_joined."ORIG_MA" AS "ORIG_MA", transactions_joined."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions_joined."DEST_STATE" AS "DEST_STATE", transactions_joined."DEST_MA" AS "DEST_MA", transactions_joined."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions_joined."NAICS" AS "NAICS", transactions_joined."QUARTER" AS "QUARTER", transactions_joined."SCTG" AS "SCTG", transactions_joined."MODE" AS "MODE", transactions_joined."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN" AS "TEMP_CNTL_YN", transactions_joined."EXPORT_YN" AS "EXPORT_YN", transactions_joined."EXPORT_CNTRY" AS "EXPORT_CNTRY", transaction

INFO:sqlalchemy.engine.base.Engine:SELECT "SHIPMT_VALUE" 
FROM (SELECT transactions_joined."SHIPMT_ID" AS "SHIPMT_ID", transactions_joined."ORIG_STATE" AS "ORIG_STATE", transactions_joined."ORIG_MA" AS "ORIG_MA", transactions_joined."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions_joined."DEST_STATE" AS "DEST_STATE", transactions_joined."DEST_MA" AS "DEST_MA", transactions_joined."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions_joined."NAICS" AS "NAICS", transactions_joined."QUARTER" AS "QUARTER", transactions_joined."SCTG" AS "SCTG", transactions_joined."MODE" AS "MODE", transactions_joined."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN" AS "TEMP_CNTL_YN", transactions_joined."EXPORT_YN" AS "EXPORT_YN", transactions_joined."EXPORT_CNTRY" AS "EXPORT_CNTRY", transactions_joined."HAZMAT" AS "HA

2016-02-08 21:15:53,893 INFO sqlalchemy.engine.base.Engine {'dest_code': 'WA', 'orig_code': 'TX'}


INFO:sqlalchemy.engine.base.Engine:{'dest_code': 'WA', 'orig_code': 'TX'}


2016-02-08 21:15:53,949 INFO sqlalchemy.engine.base.Engine ROLLBACK


INFO:sqlalchemy.engine.base.Engine:ROLLBACK


ProgrammingError: (psycopg2.ProgrammingError) subquery in FROM must have an alias
LINE 2: FROM (SELECT transactions_joined."SHIPMT_ID" AS "SHIPMT_ID",...
             ^
HINT:  For example, FROM (SELECT ...) [AS] foo.
 [SQL: 'SELECT "SHIPMT_VALUE" \nFROM (SELECT transactions_joined."SHIPMT_ID" AS "SHIPMT_ID", transactions_joined."ORIG_STATE" AS "ORIG_STATE", transactions_joined."ORIG_MA" AS "ORIG_MA", transactions_joined."ORIG_CFS_AREA" AS "ORIG_CFS_AREA", transactions_joined."DEST_STATE" AS "DEST_STATE", transactions_joined."DEST_MA" AS "DEST_MA", transactions_joined."DEST_CFS_AREA" AS "DEST_CFS_AREA", transactions_joined."NAICS" AS "NAICS", transactions_joined."QUARTER" AS "QUARTER", transactions_joined."SCTG" AS "SCTG", transactions_joined."MODE" AS "MODE", transactions_joined."SHIPMT_VALUE" AS "SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT" AS "SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC" AS "SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED" AS "SHIPMT_DIST_ROUTED", 

##### Controlling correlations

You can use the correlate method to control the behavior of correlations

In [63]:
import json

Now you can export to a json file for a temporary data file

In [64]:
#with open('app/testdata.json', 'w') as f:
#    f.write((json.dumps(list_of_dicts, indent=2, separators=(',', ':'))))

In [65]:
c = case([(transactions.c['ORIG_STATE']==transactions.c['DEST_STATE'], True)], else_=False)

In [66]:
s = select([transactions]).where(c).limit(10);

In [67]:
for i in get_rows(conn.execute(s).fetchall()):
    print(i)

2016-02-08 21:16:29,335 INFO sqlalchemy.engine.base.Engine SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
WHERE CASE WHEN (transactions."ORIG_STATE" = transactions."DEST_STATE") THEN %(param_1)s ELSE %(param_2)s END 
 LIMIT %(param_3)s


INFO:sqlalchemy.engine.base.Engine:SELECT transactions."SHIPMT_ID", transactions."ORIG_STATE", transactions."ORIG_MA", transactions."ORIG_CFS_AREA", transactions."DEST_STATE", transactions."DEST_MA", transactions."DEST_CFS_AREA", transactions."NAICS", transactions."QUARTER", transactions."SCTG", transactions."MODE", transactions."SHIPMT_VALUE", transactions."SHIPMT_WGHT", transactions."SHIPMT_DIST_GC", transactions."SHIPMT_DIST_ROUTED", transactions."TEMP_CNTL_YN", transactions."EXPORT_YN", transactions."EXPORT_CNTRY", transactions."HAZMAT", transactions."WGT_FACTOR" 
FROM transactions 
WHERE CASE WHEN (transactions."ORIG_STATE" = transactions."DEST_STATE") THEN %(param_1)s ELSE %(param_2)s END 
 LIMIT %(param_3)s


2016-02-08 21:16:29,337 INFO sqlalchemy.engine.base.Engine {'param_1': True, 'param_2': False, 'param_3': 10}


INFO:sqlalchemy.engine.base.Engine:{'param_1': True, 'param_2': False, 'param_3': 10}


{'HAZMAT': 'N', 'ORIG_MA': 148, 'SHIPMT_DIST_ROUTED': 17, 'SHIPMT_VALUE': 2178, 'MODE': 14, 'SHIPMT_DIST_GC': 14, 'ORIG_STATE': 25, 'ORIG_CFS_AREA': '25-148', 'SHIPMT_WGHT': 11, 'SCTG': '35', 'WGT_FACTOR': 208.5, 'SHIPMT_ID': 1, 'EXPORT_CNTRY': 'N', 'TEMP_CNTL_YN': 'N', 'DEST_STATE': 25, 'NAICS': 333, 'QUARTER': 2, 'DEST_CFS_AREA': '25-148', 'DEST_MA': 148, 'EXPORT_YN': 'N'}
{'HAZMAT': 'N', 'ORIG_MA': 556, 'SHIPMT_DIST_ROUTED': 3, 'SHIPMT_VALUE': 116, 'MODE': 4, 'SHIPMT_DIST_GC': 3, 'ORIG_STATE': 20, 'ORIG_CFS_AREA': '20-556', 'SHIPMT_WGHT': 6, 'SCTG': '29', 'WGT_FACTOR': 238.7, 'SHIPMT_ID': 4, 'EXPORT_CNTRY': 'N', 'TEMP_CNTL_YN': 'N', 'DEST_STATE': 20, 'NAICS': 323, 'QUARTER': 1, 'DEST_CFS_AREA': '20-556', 'DEST_MA': 556, 'EXPORT_YN': 'N'}
{'HAZMAT': 'N', 'ORIG_MA': 99999, 'SHIPMT_DIST_ROUTED': 201, 'SHIPMT_VALUE': 388, 'MODE': 5, 'SHIPMT_DIST_GC': 124, 'ORIG_STATE': 12, 'ORIG_CFS_AREA': '12-99999', 'SHIPMT_WGHT': 527, 'SCTG': '33', 'WGT_FACTOR': 398.1, 'SHIPMT_ID': 5, 'EXPORT_CNTRY':

Data structure

{orig_state_code:
 dest_state_code:
 orig_coords:
 dest_coords:
}

In [68]:
joined_states = text(
    """
    SELECT 
        fips."Name" as state_name,
        fips."Numeric code" as numeric_code,
        state_coords.latitude as lat,
        state_coords.longitude as long
    FROM "FIPS_States" fips
    JOIN "state_latlon" state_coords
    ON state_coords.state = fips."Alpha code"
    """)

list(get_rows(conn.execute(joined_states).fetchall()))

2016-02-08 21:16:56,732 INFO sqlalchemy.engine.base.Engine 
    SELECT 
        fips."Name" as state_name,
        fips."Numeric code" as numeric_code,
        state_coords.latitude as lat,
        state_coords.longitude as long
    FROM "FIPS_States" fips
    JOIN "state_latlon" state_coords
    ON state_coords.state = fips."Alpha code"
    


INFO:sqlalchemy.engine.base.Engine:
    SELECT 
        fips."Name" as state_name,
        fips."Numeric code" as numeric_code,
        state_coords.latitude as lat,
        state_coords.longitude as long
    FROM "FIPS_States" fips
    JOIN "state_latlon" state_coords
    ON state_coords.state = fips."Alpha code"
    


2016-02-08 21:16:56,733 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


[{'lat': 32.799, 'long': -86.8073, 'numeric_code': 1, 'state_name': 'Alabama'},
 {'lat': 61.385, 'long': -152.2683, 'numeric_code': 2, 'state_name': 'Alaska'},
 {'lat': 14.2417,
  'long': -170.7197,
  'numeric_code': 60,
  'state_name': 'American Samoa'},
 {'lat': 33.7712,
  'long': -111.3877,
  'numeric_code': 4,
  'state_name': 'Arizona'},
 {'lat': 34.9513,
  'long': -92.3809,
  'numeric_code': 5,
  'state_name': 'Arkansas'},
 {'lat': 36.17,
  'long': -119.7462,
  'numeric_code': 6,
  'state_name': 'California'},
 {'lat': 39.0646,
  'long': -105.3272,
  'numeric_code': 8,
  'state_name': 'Colorado'},
 {'lat': 41.5834,
  'long': -72.7622,
  'numeric_code': 9,
  'state_name': 'Connecticut'},
 {'lat': 39.3498,
  'long': -75.5148,
  'numeric_code': 10,
  'state_name': 'Delaware'},
 {'lat': 38.8964,
  'long': -77.0262,
  'numeric_code': 11,
  'state_name': 'District of Columbia'},
 {'lat': 27.8333,
  'long': -81.717,
  'numeric_code': 12,
  'state_name': 'Florida'},
 {'lat': 32.9866,
  'l

In [69]:
states_and_coords = select([state_latlon, FIPS_States]).where(state_latlon.c.state == FIPS_States.c.get('Alpha code'))

In [70]:
conn.execute(states_and_coords)

2016-02-08 21:17:01,245 INFO sqlalchemy.engine.base.Engine SELECT state_latlon.state, state_latlon.latitude, state_latlon.longitude, "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM state_latlon, "FIPS_States" 
WHERE state_latlon.state = "FIPS_States"."Alpha code"


INFO:sqlalchemy.engine.base.Engine:SELECT state_latlon.state, state_latlon.latitude, state_latlon.longitude, "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM state_latlon, "FIPS_States" 
WHERE state_latlon.state = "FIPS_States"."Alpha code"


2016-02-08 21:17:01,246 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


In [71]:
orig_state = 'CA'
dest_state = 'TX'

In [72]:
states = select([FIPS_States]).where(or_(
    FIPS_States.c['Alpha code']==orig_state,
    FIPS_States.c['Alpha code']==dest_state)
)

conn.execute(states).fetchall()

2016-02-08 21:17:03,269 INFO sqlalchemy.engine.base.Engine SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(Alpha code_1)s OR "FIPS_States"."Alpha code" = %(Alpha code_2)s


INFO:sqlalchemy.engine.base.Engine:SELECT "FIPS_States"."Name", "FIPS_States"."Alpha code", "FIPS_States"."Numeric code", "FIPS_States"."Status" 
FROM "FIPS_States" 
WHERE "FIPS_States"."Alpha code" = %(Alpha code_1)s OR "FIPS_States"."Alpha code" = %(Alpha code_2)s


2016-02-08 21:17:03,270 INFO sqlalchemy.engine.base.Engine {'Alpha code_1': 'CA', 'Alpha code_2': 'TX'}


INFO:sqlalchemy.engine.base.Engine:{'Alpha code_1': 'CA', 'Alpha code_2': 'TX'}


[('California', 'CA', 6, 'State;\xa0counties'),
 ('Texas', 'TX', 48, 'State;\xa0counties')]

### I need a way to query a database like so, 

    "SELECT * 
        FROM transactions
        WHERE orig_state = 'CA' AND dest_state= 'TX'
    "

In [ ]:
d = pd.read_sql_query(con = engine, sql = select([FIPS_States]))

In [ ]:
d2 = pd.read_sql_query(con = engine, sql = select([transactions]).limit(2000))

In [ ]:
d3 = pd.merge(pd.merge(d, d2, left_on=["Numeric code"], right_on=["ORIG_STATE"]), d, 
         left_on="DEST_STATE", right_on="Numeric code")

In [ ]:
d3 = d3.rename(columns={'Name_x': 'Source_Name', 
                   'Alpha code_x': 'Source_alpha_code',
                    'Name_y': 'Dest_Name',
                    'Alpha code_y': 'Dest_alpha_code'})

Use set operators from python to get the intersection of these predicates

In [ ]:
d3[(d3['Dest_alpha_code'] == 'CA') & (d3['Source_alpha_code'] == 'TX')]

In [ ]:
gb = d3.groupby(['Dest_alpha_code', 'Source_alpha_code'])

In [ ]:
gb.get_group(('CA', 'CA')).SHIPMT_VALUE.describe().reset_index()

In [ ]:
gb.groups.keys()

### This way works, but I'm worried that there will be a lot of overhead joining tables constantly, so I will create a new table that contains the source name, and alpha code along with the destination info into a new table called transactions_joined

### You can create a new table from a previous one using sql alchemy's Insert From Select

In [ ]:
#list(get_rows(conn.execute(select([FIPS_States.c.get("Numeric code")])).fetchall()))

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

In [ ]:
orig = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.ORIG_STATE == FIPS_States.c.get("Numeric code"))


s = select([transactions, 
            FIPS_States.c.get("Alpha code").label("orig_state_alpha_code"),
            FIPS_States.c.Name.label("orig_state_name"),
            state_latlon.c.latitude.label('orig_lat'),
            state_latlon.c.longitude.label('orig_lon')
            ]).select_from(orig).limit(10)

list(get_rows(conn.execute(s).fetchall()))

In [ ]:
dest = transactions.join(
    state_latlon_joined, 
    onclause=transactions.c.DEST_STATE == FIPS_States.c.get("Numeric code"))

new_table =  (select([s.alias(), 
            FIPS_States.c.get("Alpha code").label("dest_state_alpha_code"),
            FIPS_States.c.Name.label("dest_state_name"),
            state_latlon.c.latitude.label('dest_lat'),
            state_latlon.c.longitude.label('dest_lon')])
            .select_from(dest).limit(10))

In [ ]:
list(get_rows(conn.execute(select([new_table.alias()])).fetchall()))

In [ ]:
meta.tables['FIPS_States']

In [ ]:
t = text("""
    SELECT transactions.*,
        orig."Name" as orig_name,
        orig."Alpha code" as orig_alpha_code,
        orig."latitude" as orig_lat,
        orig."longitude" as orig_long,
        dest."Name" as dest_name,
        dest."Alpha code" as dest_alpha_code,
        dest."latitude" as dest_lat,
        dest."longitude" as dest_lon
    FROM transactions
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) orig
        ON orig."Numeric code" = transactions."ORIG_STATE"
    JOIN (
        SELECT states.*, lonlats.*
        FROM "FIPS_States" states
        JOIN state_latlon lonlats
        on states."Alpha code"=lonlats.state
         ) dest
        ON dest."Numeric code" = transactions."DEST_STATE"
    WHERE
        dest."Alpha code"=:dest_code AND orig."Alpha code" =:orig_code
    LIMIT 100
        
    """)

In [ ]:
select([transactions,
        ])

In [ ]:
conn.execute(t.params(dest_code="CA", orig_code="TX")).fetchone()

In [ ]:
#with open('test_data.json', 'w') as f:
#    f.write(
#        json.dumps(
#            list(get_rows(conn.execute(t).fetchall())),
#            indent=2,
#            separators=(",", ":")
#             ))

In [77]:
from sqlalchemy.sql import table

In [ ]:
#new_table = new_table.alias("namedStates")

In [ ]:
new_table_t = table(new_table)

In [ ]:
new_tableb

In [ ]:
orig_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.orig_state_alpha_code)

dest_ = new_table.join(
    state_latlon,
    onclause=state_latlon.c.state == new_table.c.dest_state_alpha_code)

In [ ]:
new_table2 = (select([new_table.alias(),
                    state_latlon.c.latitude.label('orig_lat'),
                    state_latlon.c.longitude.label('orig_lon')])
            .select_from(orig_.alias()))

In [ ]:
get_rows(conn.execute(select([new_table2.alias()])))

In [ ]:
list(get_rows(conn.execute(select([state_latlon])).fetchall()))

In [ ]:
list(get_rows(conn.execute(select([FIPS_States])).fetchall()))

In [ ]:
orig = state_latlon_joined.alias("orig")

dest = state_latlon_joined.alias("dest")
   

In [ ]:
orig.c.keys()

In [28]:
from sqlalchemy.sql import bindparam

In [ ]:
state_latlon_joined = FIPS_States.join(
    state_latlon, onclause=state_latlon.c.state == FIPS_States.c["Alpha code"])

stmt = (select([transactions,
              orig.c.FIPS_States_Name.label("orig_name"),
              orig.c["FIPS_States_Alpha code"].label("orig_alpha_code"),
              dest.c["FIPS_States_Name"].label("dest_name"),
              dest.c["FIPS_States_Alpha code"].label("dest_alpha_code"),
              orig.c.state_latlon_latitude.label("orig_lat"),
              orig.c.state_latlon_longitude.label("orig_lon"),
              dest.c.state_latlon_latitude.label("dest_lat"),
              dest.c.state_latlon_longitude.label("dest_lon")])
        .select_from(transactions
                    .join(orig, transactions.c.ORIG_STATE == orig.c["FIPS_States_Numeric code"])
                    .join(dest, transactions.c.DEST_STATE == dest.c["FIPS_States_Numeric code"]))
        )
        #.where(orig.c["FIPS_States_Alpha code"] == bindparam("orig_code")))
        #.where(dest.c["FIPS_States_Alpha code"] == bindparam("dest_code")))

In [ ]:
list(get_rows(conn.execute(stmt.order_by(desc(transactions.c.SHIPMT_VALUE)).limit(10), orig_code='CA').fetchall()))

Create table from another table


In [ ]:
stmt.columns.items()

In [ ]:
# a way to copy a columns from one table to another
t = Table("transactions_joined", meta, *(map(lambda c: c.copy(), stmt.columns)))


In [ ]:
t.create(bind=engine)

In [ ]:
# pass in the column keys and the statement to select from
conn.execute(t.insert().from_select(stmt.columns.keys(), stmt))

In [ ]:
meta.reflect(engine)

In [ ]:
meta.tables.keys()

In [ ]:
tj = meta.tables['transactions_joined']

In [ ]:
q = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
     .where(tj.c.dest_alpha_code==bindparam("dest_code"))
     .order_by(tj.c.SHIPMT_VALUE)
     .limit(10))

q2 = (select([tj])
     .where(tj.c.orig_alpha_code==bindparam("orig_code"))
      .order_by(desc(tj.c.SHIPMT_VALUE))
      .limit(10))
list(get_rows(conn.execute(q, orig_code="CA", dest_code="TX")))

In [ ]:
tj.columns.keys()

In [ ]:
from sqlalchemy import Index

Index('orig_fip', transactions.c.ORIG_STATE)

In [ ]:
import timeit

I want to experiment with adding an index and testing the response of the query

In [ ]:
import time

In [ ]:
print("We will be running: {}".format(str(q)))

We need to make an index on the orig_alpha_code and dest_alpha_code

In [ ]:
index = text("CREATE INDEX ALPHA_INDEX ON transactions_joined "\
             "(orig_alpha_code, dest_alpha_code)")

In [ ]:
conn.execute(index)

In [ ]:
meta.tables['transactions_joined']

In [29]:
#http://stackoverflow.com/questions/5605019/listing-indices-using-sqlalchemy
from sqlalchemy.engine import reflection

In [30]:
insp = reflection.Inspector.from_engine(engine)
indices = []
for name in insp.get_table_names():
    for index in insp.get_indexes(name):
        indices.append((name,index))

2016-01-09 00:05:17,651 INFO sqlalchemy.engine.base.Engine SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


INFO:sqlalchemy.engine.base.Engine:SELECT relname FROM pg_class c WHERE relkind = 'r' AND 'public' = (select nspname from pg_namespace n where n.oid = c.relnamespace) 


2016-01-09 00:05:17,653 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-01-09 00:05:17,789 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in ('r', 'v', 'm', 'f')
        


...

INFO:sqlalchemy.engine.base.Engine:{'table_name': 'naics'}


2016-01-09 00:05:19,766 INFO sqlalchemy.engine.base.Engine 
              SELECT
                  i.relname as relname,
                  ix.indisunique, ix.indexprs, ix.indpred,
                  a.attname, a.attnum, c.conrelid, ix.indkey::varchar,
                  i.reloptions, am.amname
              FROM
                  pg_class t
                        join pg_index ix on t.oid = ix.indrelid
                        join pg_class i on i.oid = ix.indexrelid
                        left outer join
                            pg_attribute a
                            on t.oid = a.attrelid and a.attnum = ANY(ix.indkey)
                        left outer join
                            pg_constraint c
                            on (ix.indrelid = c.conrelid and
                                ix.indexrelid = c.conindid and
                                c.contype in ('p', 'u', 'x'))
                        left outer join
                            pg_am am
                    

INFO:sqlalchemy.engine.base.Engine:
              SELECT
                  i.relname as relname,
                  ix.indisunique, ix.indexprs, ix.indpred,
                  a.attname, a.attnum, c.conrelid, ix.indkey::varchar,
                  i.reloptions, am.amname
              FROM
                  pg_class t
                        join pg_index ix on t.oid = ix.indrelid
                        join pg_class i on i.oid = ix.indexrelid
                        left outer join
                            pg_attribute a
                            on t.oid = a.attrelid and a.attnum = ANY(ix.indkey)
                        left outer join
                            pg_constraint c
                            on (ix.indrelid = c.conrelid and
                                ix.indexrelid = c.conindid and
                                c.contype in ('p', 'u', 'x'))
                        left outer join
                            pg_am am
                            on i.relam = am.

2016-01-09 00:05:19,767 INFO sqlalchemy.engine.base.Engine {'table_oid': 16442}


INFO:sqlalchemy.engine.base.Engine:{'table_oid': 16442}


In [31]:
indices

[('transactions_joined',
  {'column_names': ['SHIPMT_VALUE'],
   'name': 'shipmt_value_desc',
   'unique': False}),
 ('transactions_joined',
  {'column_names': ['SHIPMT_VALUE'], 'name': 'shipmt_value', 'unique': False}),
 ('transactions_joined',
  {'column_names': ['orig_alpha_code', 'dest_alpha_code'],
   'name': 'alpha_index',
   'unique': False})]

In [ ]:
import datetime

In [ ]:
now = datetime.datetime.now()
l = list(get_rows(conn.execute(q, orig_code="NY", dest_code="TX").fetchall()))
diff = datetime.datetime.now() - now



In [ ]:
print("It took {} seconds to execute the query".format(diff))
pprint(l)

In [32]:
#http://docs.sqlalchemy.org/en/latest/faq/performance.html#query-profiling
from sqlalchemy import event
from sqlalchemy.engine import Engine

In [33]:
@event.listens_for(Engine, "before_cursor_execute")
def before_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    conn.info.setdefault('query_start_time', []).append(time.time())
    print("Start Query: {}".format(statement))

In [34]:
@event.listens_for(Engine, "after_cursor_execute")
def after_cursor_execute(conn, cursor, statement, parameters, context, executemany):
    total = time.time() - conn.info['query_start_time'].pop(-1)
    print("Query Done\nTotal-Time: {}".format(total))

In [ ]:
l = list(get_rows(conn.execute(q2, orig_code="CA").fetchall()))

In [ ]:
cmp = q.params(orig_code="CA", dest_code="TX").compile()

In [ ]:
print(cmp)

In [ ]:
# try creating a new index on shipmt_value
i = Index('shipmt_value', tj.c.SHIPMT_VALUE)
i.create(engine)

In [ ]:
l = list(get_rows(conn.execute(q, orig_code="WY", dest_code="WY").fetchall()))

In [ ]:
# that index made it much faster
# let's try adding a functional index as well, a desc version of shipmt_value
i = Index('shipmt_value_desc', tj.c.SHIPMT_VALUE.desc())
i.create(engine)

### Adding indexes greatly reduces time!

## Now I will be figuring out the queries needed for the other data views

In [ ]:
# we want to get the sctg names joined
q = (select([tj.c.SCTG, func.count(tj.c.SHIPMT_ID).label("counts")])
     .where(and_(
        tj.c.dest_alpha_code == 'TX',
        tj.c.orig_alpha_code == 'WY'))
     )
        

In [ ]:
g1 = q.group_by(tj.c.SCTG)

In [ ]:
for i in get_rows(conn.execute(g1.limit(10)).fetchall()):
    print(i)

In [ ]:
sctg.columns.keys()

In [ ]:
for i in get_rows(conn.execute(select([g1.alias(), sctg]).where(tj.c.SCTG == cast(sctg.c.SCTG, TEXT()))).fetchall()):
    print(i)

In [ ]:
from sqlalchemy import cast, Numeric, String, TEXT

In [ ]:
tj.c.SCTG, print(cast(sctg.c.SCTG, TEXT()))

In [27]:
transactionsBetweenStates = (select([transactions_joined])
                                 .where(
                transactions_joined.c.orig_alpha_code==bindparam("orig_code"))
                                  .where(
                transactions_joined.c.dest_alpha_code==bindparam("dest_code"))
)




In [28]:
q = transactionsBetweenStates.params(orig_code='CA', dest_code='CA')

In [34]:
_data = pd.read_sql_query(q, engine)

2016-01-15 19:22:55,809 INFO sqlalchemy.engine.base.Engine SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM 

INFO:sqlalchemy.engine.base.Engine:SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM transactions_joined 
WHE

2016-01-15 19:22:55,811 INFO sqlalchemy.engine.base.Engine {'dest_code': 'CA', 'orig_code': 'CA'}


INFO:sqlalchemy.engine.base.Engine:{'dest_code': 'CA', 'orig_code': 'CA'}


In [51]:
_data['QUARTER'].value_counts

Object `value_counts` not found.


In [56]:
sser= _data['QUARTER'].value_counts(

In [61]:
import json

In [68]:
json.dumps([{str(k):str(v)} for k,v in sser.sort_index().to_dict().items()])

'[{"1": "58524"}, {"2": "58017"}, {"3": "55253"}, {"4": "50051"}]'

In [76]:
conn.execute(select([transactions_joined]).limit(10)).fetchall()

2016-02-08 21:19:31,495 INFO sqlalchemy.engine.base.Engine SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM 

INFO:sqlalchemy.engine.base.Engine:SELECT transactions_joined."SHIPMT_ID", transactions_joined."ORIG_STATE", transactions_joined."ORIG_MA", transactions_joined."ORIG_CFS_AREA", transactions_joined."DEST_STATE", transactions_joined."DEST_MA", transactions_joined."DEST_CFS_AREA", transactions_joined."NAICS", transactions_joined."QUARTER", transactions_joined."SCTG", transactions_joined."MODE", transactions_joined."SHIPMT_VALUE", transactions_joined."SHIPMT_WGHT", transactions_joined."SHIPMT_DIST_GC", transactions_joined."SHIPMT_DIST_ROUTED", transactions_joined."TEMP_CNTL_YN", transactions_joined."EXPORT_YN", transactions_joined."EXPORT_CNTRY", transactions_joined."HAZMAT", transactions_joined."WGT_FACTOR", transactions_joined.orig_name, transactions_joined.orig_alpha_code, transactions_joined.dest_name, transactions_joined.dest_alpha_code, transactions_joined.orig_lat, transactions_joined.orig_lon, transactions_joined.dest_lat, transactions_joined.dest_lon 
FROM transactions_joined 
 LI

2016-02-08 21:19:31,497 INFO sqlalchemy.engine.base.Engine {'param_1': 10}


INFO:sqlalchemy.engine.base.Engine:{'param_1': 10}


[(3430894, 4, 536, '04-536', 4, 99999, '04-99999', 212, 1, '14', 6, 147651, 180762, 106, 269, 'N', 'N', 'N', 'N', 455.0, 'Arizona', 'AZ', 'Arizona', 'AZ', 33.7712, -111.3877, 33.7712, -111.3877),
 (3430895, 41, 99999, '41-99999', 41, 440, '41-440', 4235, 1, '32', 5, 5817, 9434, 94, 121, 'N', 'N', 'N', 'N', 243.3, 'Oregon', 'OR', 'Oregon', 'OR', 44.5672, -122.1269, 44.5672, -122.1269),
 (3430896, 48, 288, '48-288', 48, 288, '48-288', 4231, 3, '24', 5, 208, 75, 28, 35, 'N', 'N', 'N', 'N', 6615.5, 'Texas', 'TX', 'Texas', 'TX', 31.106, -97.6475, 31.106, -97.6475),
 (3430897, 6, 348, '06-348', 6, 260, '06-260', 4243, 1, '40', 14, 1025, 53, 210, 226, 'N', 'N', 'N', 'N', 9193.1, 'California', 'CA', 'California', 'CA', 36.17, -119.7462, 36.17, -119.7462),
 (3430898, 17, 476, '17-476', 29, 99999, '29-99999', 311, 4, '07', 4, 10180, 43346, 176, 215, 'Y', 'N', 'N', 'N', 140.9, 'Illinois', 'IL', 'Missouri', 'MO', 40.3363, -89.0022, 38.4623, -92.302),
 (3430899, 33, 99999, '33-99999', 33, 99999, '3

In [90]:
conn.execute(text(
        """
        SELECT "orig_alpha_code", 
                COUNT(1) as shipment_counts
        FROM   transactions_joined
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
        """
    )).fetchall()

2016-02-08 21:48:46,293 INFO sqlalchemy.engine.base.Engine 
        SELECT "orig_alpha_code", 
                COUNT(1) as shipment_counts
        FROM   transactions_joined
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
        


INFO:sqlalchemy.engine.base.Engine:
        SELECT "orig_alpha_code", 
                COUNT(1) as shipment_counts
        FROM   transactions_joined
        GROUP BY 1
        ORDER BY 2 DESC
        LIMIT 10
        


2016-02-08 21:48:46,294 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


[('CA', 341984),
 ('TX', 267822),
 ('OH', 221321),
 ('PA', 205959),
 ('NY', 184429),
 ('FL', 172342),
 ('IL', 170608),
 ('MI', 150136),
 ('NC', 148681),
 ('IN', 141060)]